<a href="https://colab.research.google.com/github/makkimaki/pytorch_official_tutorial/blob/main/pytorch_tutorial_4_build_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
import torch 
from torch import nn 
from torch.utils.data import DataLoader 
from torchvision import datasets, transforms 

## Get device for training

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


# Define the Class

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x 


In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


ここでインスタンス化した`model`にデータを渡せば勝手に`forward`メソッドまでが実行される．`model.forward()`とはしないように！

In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)

print(f"Predicted class: {y_pred}")

Predicted class: tensor([0])


各層で何が起きているかを見ていこう

In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten
2D 28*28の画像を隣接する784ピクセルの値に変換する．
（この時，ミニバッチ次元(dim=0)は保存されたまま）

In [12]:
flatten = nn.Flatten()
flat_image = flatten(input_image)

print(flat_image.shape)

print(flat_image.size)
print(flat_image.size())

torch.Size([3, 784])
<built-in method size of Tensor object at 0x7f7fc5d4a460>
torch.Size([3, 784])


## nn.Linear


In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU

In [16]:
print(f"Before ReLU \n: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU \n: {hidden1}")

Before ReLU 
: tensor([[0.0000, 0.3708, 0.3230, 0.2592, 0.2365, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.2311, 0.1798, 0.4547, 0.0000, 0.4527, 0.0000, 0.0044, 0.3616,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.5207, 0.5377, 0.3597, 0.0265, 0.2986, 0.0038, 0.0000,
         0.0000, 0.6516, 0.1145, 0.7049, 0.0000, 0.0892, 0.0000, 0.3658, 0.2842,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.1479, 0.2185, 0.0423, 0.0000, 0.1138, 0.0399, 0.0000,
         0.0000, 0.1341, 0.1158, 0.4120, 0.0000, 0.3418, 0.1692, 0.0780, 0.3004,
         0.0000, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU 
: tensor([[0.0000, 0.3708, 0.3230, 0.2592, 0.2365, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.2311, 0.1798, 0.4547, 0.0000, 0.4527, 0.0000, 0.0044, 0.3616,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.5207, 0.5377, 0.3597, 0.0265, 0.2986, 0.0038, 0.0000,
         0.0000, 0.6516, 0.1145, 0.7049, 0.0000, 0.0892, 0.0000, 0.3658, 0.2842,
         0.0000, 0.0000],
   

## nn.Sequential

In [18]:
seq_modules = nn.Sequential(
    flatten, 
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

## nn.Softmax
`dim`パラメータは，確率の和が１になるように沿う次元の総数を表す．

In [19]:
softmax = nn.Softmax(dim=1)
pred_prob = softmax(logits)